# Combined RAG System: Intent Parser + Medical RAG + Nutrition RAG

A unified nutrition recommendation system that combines medical knowledge with food database search.

## System Flow

```
┌────────────────────┐
│    USER QUERY      │  "I have diabetes, want chicken and eggs, allergic to tomatoes"
└─────────┬──────────┘
          │
          ▼
┌────────────────────┐
│ 1. INTENT PARSER   │  → Extracts: conditions, ingredients, allergies, style
└─────────┬──────────┘
          │
          ▼
┌────────────────────┐
│ 2. MEDICAL RAG     │  → Returns: nutrition constraints from PDF knowledge base
│    (PDFs)          │     {sugar_g: max 10, sodium_mg: max 1500, fiber_g: min 25}
└─────────┬──────────┘
          │
          ▼
┌────────────────────┐
│ 3. AUGMENTED QUERY │  → Combines: user request + medical constraints + allergies
│    BUILDER         │     into a rich prompt for the LLM
└─────────┬──────────┘
          │
          ▼
┌────────────────────┐
│ 4. NUTRITION RAG   │  → LLM generates recommendations using augmented query
│    (Food Database) │     with full constraint awareness
└─────────┬──────────┘
          │
          ▼
┌────────────────────┐
│ 5. SAFETY CHECK    │  → Filters out: allergens, foods exceeding limits,
│                    │     foods on "avoid" list
└─────────┬──────────┘
          │
          ▼
┌────────────────────┐
│ 6. FINAL RESULT    │  → LLM recommendation + verified safe ingredients
└────────────────────┘
```

## Components

| Component | Input | Output |
|-----------|-------|--------|
| Intent Parser | Natural language query | Structured intent (conditions, allergies, ingredients) |
| Medical RAG | Medical conditions | Nutrition constraints + foods to avoid |
| Query Builder | Intent + Constraints | Augmented prompt with all context |
| Nutrition RAG | Augmented query | Food recommendations with explanations |
| Safety Check | Candidate foods | Filtered safe foods only |

## 1. Setup and Imports

In [13]:
# Core imports
import json
import re
import os
from pathlib import Path
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field

# LangChain imports
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_core.output_parsers import JsonOutputParser

# Ollama LLM
from langchain_ollama import OllamaLLM

# Data processing
import pandas as pd
import numpy as np

print("All imports successful!")

All imports successful!


In [14]:
# Configuration paths
PDF_FOLDER = "../data/pdfs/"
NUTRITION_DATA_PATH = "../data/raw/nutrition.xlsx"
MEDICAL_VECTORSTORE_PATH = "../data/processed/medical_pdfs_vectorstore"
NUTRITION_VECTORSTORE_PATH = "../data/processed/nutrition_vectorstore"

# LLM Configuration
LLM_MODEL = "llama3.2"

print(f"PDF folder: {PDF_FOLDER}")
print(f"Nutrition data: {NUTRITION_DATA_PATH}")

PDF folder: ../data/pdfs/
Nutrition data: ../data/raw/nutrition.xlsx


---
## 2. Intent Parser Component

Parses user query to extract fields aligned with the User model:
- `name`: user's name if mentioned
- `surname`: user's surname if mentioned
- `preferences`: ingredients, cooking style, cuisine preferences
- `restrictions`: dietary restrictions (keto, vegan, low_sodium, etc.)
- `health_condition`: medical conditions (diabetes, hypertension, parkinsons, etc.)
- `caretaker`: caretaker name if mentioned

Fields not found in the query are left as empty strings.

In [15]:
@dataclass
class UserIntent:
    """Structured representation of parsed user intent.
    
    Aligned with the User database model fields.
    Fields not found in the query remain as empty strings.
    """
    name: str = ""
    surname: str = ""
    preferences: str = ""       # ingredients, cooking style, cuisine
    restrictions: str = ""      # dietary restrictions (keto, vegan, etc.)
    health_condition: str = ""  # medical conditions (diabetes, hypertension, etc.)
    caretaker: str = ""
    
    # Parsed lists used internally by the pipeline (not stored in DB)
    allergies: List[str] = field(default_factory=list)
    
    def __repr__(self):
        fields = {
            "name": self.name,
            "surname": self.surname,
            "preferences": self.preferences,
            "restrictions": self.restrictions,
            "health_condition": self.health_condition,
            "caretaker": self.caretaker,
            "allergies": self.allergies,
        }
        lines = [f"  {k}: {v}" for k, v in fields.items() if v]
        return "UserIntent:\n" + "\n".join(lines) if lines else "UserIntent: (empty)"
    
    @property
    def medical_conditions_list(self) -> List[str]:
        """Return health_condition as a list for pipeline processing."""
        return [c.strip() for c in self.health_condition.split(",") if c.strip()]
    
    @property
    def ingredients_list(self) -> List[str]:
        """Extract ingredient names from preferences string."""
        return [p.strip() for p in self.preferences.split(",") if p.strip()]


class IntentParser:
    """Parses user queries to extract structured intent using LLM."""
    
    def __init__(self, model_name: str = "llama3.2"):
        self.llm = OllamaLLM(
            model=model_name,
            temperature=0,
            format="json"
        )
        self.parser = JsonOutputParser()
        self.chain = self._build_chain()
    
    def _build_chain(self):
        system_instructions = """You are a medical nutrition data extractor.
Your job: read the user query and extract ONLY information that is explicitly stated.
Do NOT add, guess, or infer anything that the user did not say.

Return a JSON object with these keys:
- "name" (string): user's first name, or "" if not stated
- "surname" (string): user's last name, or "" if not stated
- "preferences" (string): foods and cooking style the user wants, comma-separated, or "" if not stated
- "restrictions" (string): dietary restrictions the user follows, comma-separated, or "" if not stated
- "health_condition" (string): medical conditions the user has, comma-separated, or "" if not stated. Use snake_case. Normalize common phrases: "high blood pressure" -> "hypertension", "sugar problem" -> "diabetes"
- "caretaker" (string): caretaker name, or "" if not stated
- "allergies" (array of strings): foods the user cannot eat or is allergic to, or [] if not stated

CRITICAL RULES:
1. ONLY extract what the user explicitly says. Never fill in a field the user did not mention.
2. If the user does not mention a field, return "" for strings or [] for arrays.
3. Do not confuse allergies with preferences. "I can't eat X" = allergy. "I want X" = preference.

EXAMPLE:
Input: "I'm John. I have diabetes. I want chicken and rice. I'm allergic to peanuts."
Output: {{"name": "John", "surname": "", "preferences": "chicken, rice", "restrictions": "", "health_condition": "diabetes", "caretaker": "", "allergies": ["peanuts"]}}

EXAMPLE:
Input: "Quick vegan lunch with tofu"
Output: {{"name": "", "surname": "", "preferences": "tofu, quick lunch", "restrictions": "vegan", "health_condition": "", "caretaker": "", "allergies": []}}"""
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_instructions),
            ("user", "{query}")
        ])
        
        return prompt | self.llm | self.parser
    
    def parse(self, query: str) -> UserIntent:
        """Parse user query and return structured intent."""
        try:
            result = self.chain.invoke({"query": query})
            return UserIntent(
                name=result.get("name", ""),
                surname=result.get("surname", ""),
                preferences=result.get("preferences", ""),
                restrictions=result.get("restrictions", ""),
                health_condition=result.get("health_condition", ""),
                caretaker=result.get("caretaker", ""),
                allergies=result.get("allergies", []),
            )
        except Exception as e:
            print(f"Error parsing intent: {e}")
            return UserIntent()


# Initialize Intent Parser
intent_parser = IntentParser(model_name=LLM_MODEL)
print("Intent Parser initialized!")

Intent Parser initialized!


In [16]:
# Test the Intent Parser
test_query = "I have parkinson. I want to make something with chicken, eggs and salad. Quick breakfast. I can't eat tomatoes or onions."

print(f"Query: {test_query}\n")
intent = intent_parser.parse(test_query)
print(intent)

Query: I have parkinson. I want to make something with chicken, eggs and salad. Quick breakfast. I can't eat tomatoes or onions.

UserIntent:
  preferences: chicken, eggs, salad
  health_condition: parkinson
  allergies: ['tomatoes', 'onions']


---
## 3. Medical RAG Component (Dummy)

Dummy implementation that returns hardcoded constraints based on condition keywords.
Replace with the real `MedicalRAG` from `src/rag/medical_rag.py` when ready.

In [17]:
class MedicalRAG:
    """Dummy Medical RAG — returns hardcoded constraints without loading PDFs."""

    def __init__(self, **kwargs):
        """Accepts any kwargs so the interface matches the real MedicalRAG."""
        pass

    def initialize(self, force_rebuild: bool = False):
        print("MedicalRAG (dummy) initialized — no PDFs loaded.")

    def get_constraints(self, conditions: List[str]) -> Dict[str, Any]:
        """Return nutrition constraints based on condition keywords."""
        constraints = {
            "constraints": {
                "sugar_g": {"max": 25},
                "sodium_mg": {"max": 2300},
                "fiber_g": {"min": 5},
                "saturated_fat_g": {"max": 20},
            },
            "increase": ["vegetables", "whole grains", "lean proteins"],
            "limit": ["processed foods", "added sugars", "salt"],
            "avoid": [],
            "notes": [],
        }

        conditions_lower = [c.lower() for c in conditions]

        if any("diabetes" in c for c in conditions_lower):
            constraints["constraints"]["sugar_g"]["max"] = 10
            constraints["avoid"].append("refined sugars")
            constraints["notes"].append("Monitor blood glucose levels")

        if any("hypertension" in c or "blood_pressure" in c for c in conditions_lower):
            constraints["constraints"]["sodium_mg"]["max"] = 1500
            constraints["notes"].append("Limit sodium intake")

        if any("parkinson" in c for c in conditions_lower):
            constraints["increase"].append("protein")
            constraints["notes"].append("Consider protein timing with medications")

        return constraints

    def ask(self, query: str) -> str:
        return "(dummy) No medical documents loaded."


# Initialize dummy Medical RAG
medical_rag = MedicalRAG()
medical_rag.initialize()
print("Medical RAG (dummy) ready!")

MedicalRAG (dummy) initialized — no PDFs loaded.
Medical RAG (dummy) ready!


---
## 4. Nutrition RAG Component (Dummy)

Dummy implementation that returns a canned LLM response and fake food documents.
Replace with the real `NutritionRAG` when ready.

In [18]:
class NutritionRAG:
    """Dummy Nutrition RAG — returns fake food documents and canned answers."""

    # Sample food entries used by search()
    _SAMPLE_FOODS = [
        {"name": "Chicken Breast", "calories": 165, "protein_g": 31, "carbs_g": 0, "fat_g": 3.6, "fiber_g": 0, "sugar_g": 0, "sodium_mg": 74},
        {"name": "Egg (boiled)", "calories": 78, "protein_g": 6, "carbs_g": 0.6, "fat_g": 5.3, "fiber_g": 0, "sugar_g": 0.6, "sodium_mg": 62},
        {"name": "Mixed Salad Greens", "calories": 20, "protein_g": 1.5, "carbs_g": 3.5, "fat_g": 0.3, "fiber_g": 2, "sugar_g": 1, "sodium_mg": 25},
        {"name": "Tomato", "calories": 18, "protein_g": 0.9, "carbs_g": 3.9, "fat_g": 0.2, "fiber_g": 1.2, "sugar_g": 2.6, "sodium_mg": 5},
        {"name": "Brown Rice", "calories": 216, "protein_g": 5, "carbs_g": 45, "fat_g": 1.8, "fiber_g": 3.5, "sugar_g": 0.7, "sodium_mg": 10},
        {"name": "Salmon Fillet", "calories": 208, "protein_g": 20, "carbs_g": 0, "fat_g": 13, "fiber_g": 0, "sugar_g": 0, "sodium_mg": 59},
        {"name": "Onion (raw)", "calories": 40, "protein_g": 1.1, "carbs_g": 9.3, "fat_g": 0.1, "fiber_g": 1.7, "sugar_g": 4.2, "sodium_mg": 4},
        {"name": "Broccoli", "calories": 55, "protein_g": 3.7, "carbs_g": 11, "fat_g": 0.6, "fiber_g": 5.1, "sugar_g": 2.2, "sodium_mg": 33},
        {"name": "Oatmeal", "calories": 154, "protein_g": 6, "carbs_g": 27, "fat_g": 2.5, "fiber_g": 4, "sugar_g": 1, "sodium_mg": 5},
        {"name": "Greek Yogurt", "calories": 100, "protein_g": 17, "carbs_g": 6, "fat_g": 0.7, "fiber_g": 0, "sugar_g": 4, "sodium_mg": 36},
    ]

    def __init__(self, **kwargs):
        pass

    def initialize(self, force_rebuild: bool = False):
        print("NutritionRAG (dummy) initialized — using sample food data.")

    def search(self, query: str, k: int = 20) -> List[Document]:
        """Return fake Document objects matching the query loosely."""
        query_lower = query.lower()
        docs = []
        for food in self._SAMPLE_FOODS:
            doc = Document(
                page_content=f"{food['name']}: {food['calories']} cal, {food['protein_g']}g protein",
                metadata=food,
            )
            docs.append(doc)
        return docs[:k]

    def ask(self, question: str) -> str:
        return (
            "(dummy) Based on your preferences, I recommend chicken breast with "
            "mixed salad greens and a side of brown rice. This provides a good balance "
            "of protein, fiber, and complex carbohydrates while being low in sugar and sodium."
        )


# Initialize dummy Nutrition RAG
nutrition_rag = NutritionRAG()
nutrition_rag.initialize()
print("Nutrition RAG (dummy) ready!")

NutritionRAG (dummy) initialized — using sample food data.
Nutrition RAG (dummy) ready!


---
## 5. Safety Filter

Filters food candidates by allergies, avoid-lists, and nutrition constraint limits.

In [19]:
class SafetyFilter:
    """Filters foods based on allergies, avoid-lists, and nutrition limits."""

    def __init__(self, debug: bool = False):
        self.debug = debug

    def filter(
        self,
        candidates: List[Document],
        allergies: List[str],
        constraints: Dict[str, Any],
        avoid_foods: List[str] = None,
    ) -> List[Document]:
        """Filter candidates by safety constraints."""
        avoid_foods = avoid_foods or []
        filtered = []

        allergies_lower = [a.lower() for a in allergies]
        avoid_lower = [a.lower() for a in avoid_foods]
        constraint_rules = constraints.get("constraints", {})

        if self.debug:
            print(f"\n[DEBUG] Filtering {len(candidates)} candidates")
            print(f"[DEBUG] Allergies: {allergies_lower}")
            print(f"[DEBUG] Avoid foods: {avoid_lower}")

        for doc in candidates:
            name_lower = doc.metadata.get("name", "").lower()

            # Check allergies
            if any(allergen in name_lower for allergen in allergies_lower):
                if self.debug:
                    print(f"  REJECTED (allergen): {name_lower}")
                continue

            # Check avoid list
            if any(avoid in name_lower for avoid in avoid_lower):
                if self.debug:
                    print(f"  REJECTED (avoid): {name_lower}")
                continue

            # Check nutrition limits
            rejected = False
            for nutrient, rule in constraint_rules.items():
                value = doc.metadata.get(nutrient, 0)
                if rule.get("max") and value > rule["max"]:
                    if self.debug:
                        print(f"  REJECTED (nutrition {nutrient}={value} > {rule['max']}): {name_lower}")
                    rejected = True
                    break
            if rejected:
                continue

            filtered.append(doc)

        return filtered


safety_filter = SafetyFilter(debug=True)
print("Safety Filter ready!")

Safety Filter ready!


In [20]:
@dataclass
class PipelineResult:
    """Result from the combined RAG pipeline."""
    intent: UserIntent
    constraints: Dict[str, Any]
    augmented_query: str
    llm_recommendation: str
    candidates_count: int
    filtered_count: int
    safe_foods: List[Dict[str, Any]]

    def display(self):
        """Pretty-print the pipeline result."""
        print("\n" + "=" * 60)
        print("PIPELINE RESULT")
        print("=" * 60)

        print(f"\n--- Intent ---\n{self.intent}")

        print(f"\n--- Constraints ---")
        for k, v in self.constraints.get("constraints", {}).items():
            print(f"  {k}: {v}")

        print(f"\n--- LLM Recommendation ---\n{self.llm_recommendation}")

        print(f"\n--- Safe Foods ({self.filtered_count}/{self.candidates_count} passed) ---")
        for i, food in enumerate(self.safe_foods, 1):
            print(f"  {i}. {food['name']}  |  {food['calories']} cal  |  "
                  f"P:{food['protein_g']}g  C:{food['carbs_g']}g  F:{food['fat_g']}g")

print("PipelineResult ready!")

PipelineResult ready!


---
## 6. Combined Pipeline

Orchestrates the full flow:
1. Parse user intent
2. Get medical constraints
3. Build augmented query
4. Get LLM recommendation
5. Safety check on retrieved foods
6. Return final results

In [21]:
class NutritionPipeline:
    """Combined pipeline for nutrition recommendations.
    
    Modular design — each step is a separate method that can be
    overridden, tested independently, or replaced with a different
    implementation (e.g., swap MedicalRAG for an API call).
    """
    
    def __init__(
        self,
        intent_parser: IntentParser,
        medical_rag: MedicalRAG,
        nutrition_rag: NutritionRAG,
        safety_filter: SafetyFilter
    ):
        self.intent_parser = intent_parser
        self.medical_rag = medical_rag
        self.nutrition_rag = nutrition_rag
        self.safety_filter = safety_filter
    
    # ── public entry point ──────────────────────────────────────
    
    def process(self, user_query: str, top_k: int = 10):
        """Run the full pipeline and return a PipelineResult."""
        print("\n" + "=" * 60)
        print("PROCESSING QUERY...")
        print("=" * 60)
        print(f"Query: {user_query}\n")
        
        intent       = self._step_parse_intent(user_query)
        constraints  = self._step_get_constraints(intent)
        augmented_q  = self._step_build_augmented_query(user_query, intent, constraints)
        llm_rec      = self._step_get_recommendation(augmented_q)
        safe_foods   = self._step_safety_filter(intent, constraints, top_k)
        
        return PipelineResult(
            intent=intent,
            constraints=constraints,
            augmented_query=augmented_q,
            llm_recommendation=llm_rec,
            candidates_count=safe_foods["candidates_count"],
            filtered_count=safe_foods["filtered_count"],
            safe_foods=safe_foods["results"],
        )
    
    # ── pipeline steps (each is independently testable) ─────────
    
    def _step_parse_intent(self, user_query: str) -> UserIntent:
        print("[Step 1] Intent Parser — extracting structured data...")
        intent = self.intent_parser.parse(user_query)
        print(f"  → Health conditions: {intent.health_condition or '(none)'}")
        print(f"  → Preferences: {intent.preferences or '(none)'}")
        print(f"  → Restrictions: {intent.restrictions or '(none)'}")
        print(f"  → Allergies: {intent.allergies or '(none)'}")
        return intent
    
    def _step_get_constraints(self, intent: UserIntent) -> Dict:
        print("\n[Step 2] Medical RAG — getting nutrition constraints...")
        constraints = self.medical_rag.get_constraints(intent.medical_conditions_list)
        print(f"  → Constraints: {list(constraints.get('constraints', {}).keys())}")
        print(f"  → Avoid: {constraints.get('avoid', [])}")
        print(f"  → Limit: {constraints.get('limit', [])}")
        return constraints
    
    def _step_build_augmented_query(
        self, original_query: str, intent: UserIntent, constraints: Dict
    ) -> str:
        print("\n[Step 3] Building augmented query...")
        augmented = self._build_augmented_query(original_query, intent, constraints)
        print(f"  → Query built ({len(augmented)} chars)")
        return augmented
    
    def _step_get_recommendation(self, augmented_query: str) -> str:
        print("\n[Step 4] Nutrition RAG — getting LLM recommendation...")
        rec = self.nutrition_rag.ask(augmented_query)
        print("  → Recommendation received")
        return rec
    
    def _step_safety_filter(
        self, intent: UserIntent, constraints: Dict, top_k: int
    ) -> Dict:
        print("\n[Step 5] Safety Check — filtering unsafe foods...")
        
        # Use ingredients from preferences for the search
        search_terms = intent.ingredients_list or ["healthy food"]
        search_query = " ".join(search_terms)
        
        candidates = self.nutrition_rag.search(search_query, k=50)
        print(f"  → Searched {len(candidates)} candidates")
        
        avoid_foods = constraints.get("avoid", [])
        safe_foods = self.safety_filter.filter(
            candidates=candidates,
            allergies=intent.allergies,
            constraints=constraints,
            avoid_foods=avoid_foods,
        )
        print(f"  → {len(safe_foods)} safe foods after filtering")
        
        results = [
            {
                "name": doc.metadata.get("name"),
                "calories": doc.metadata.get("calories", 0),
                "protein_g": doc.metadata.get("protein_g", 0),
                "carbs_g": doc.metadata.get("carbs_g", 0),
                "fat_g": doc.metadata.get("fat_g", 0),
                "fiber_g": doc.metadata.get("fiber_g", 0),
                "sugar_g": doc.metadata.get("sugar_g", 0),
                "sodium_mg": doc.metadata.get("sodium_mg", 0),
            }
            for doc in safe_foods[:top_k]
        ]
        
        return {
            "candidates_count": len(candidates),
            "filtered_count": len(safe_foods),
            "results": results,
        }
    
    # ── query builder ───────────────────────────────────────────
    
    def _build_augmented_query(
        self, original_query: str, intent: UserIntent, constraints: Dict
    ) -> str:
        """Build augmented query using User-model-aligned intent fields."""
        constraint_rules = constraints.get("constraints", {})
        constraint_text = []
        
        if constraint_rules.get("sugar_g", {}).get("max"):
            constraint_text.append(f"prefer foods lower in sugar (daily limit: {constraint_rules['sugar_g']['max']}g)")
        if constraint_rules.get("sodium_mg", {}).get("max"):
            constraint_text.append(f"prefer foods lower in sodium (daily limit: {constraint_rules['sodium_mg']['max']}mg)")
        if constraint_rules.get("fiber_g", {}).get("min"):
            constraint_text.append(f"prefer foods with good fiber content (daily goal: {constraint_rules['fiber_g']['min']}g)")
        if constraint_rules.get("protein_g", {}).get("max"):
            constraint_text.append(f"moderate protein intake (daily limit: {constraint_rules['protein_g']['max']}g)")
        
        sections = [f"USER REQUEST: {original_query}"]
        
        if intent.health_condition:
            sections.append(f"MEDICAL CONDITIONS: {intent.health_condition}")
        
        if intent.restrictions:
            sections.append(f"DIETARY RESTRICTIONS: {intent.restrictions}")
        
        sections.append(
            "NUTRITION GUIDELINES (daily targets):\n"
            + ("\n".join("- " + c for c in constraint_text) if constraint_text else "- General healthy eating")
        )
        
        sections.append(
            "ALLERGIES (MUST AVOID):\n"
            + ("\n".join("- " + a for a in intent.allergies) if intent.allergies else "- None")
        )
        
        avoid = constraints.get("avoid", [])
        sections.append(
            "FOODS TO AVOID:\n"
            + ("\n".join("- " + f for f in avoid) if avoid else "- None")
        )
        
        limit = constraints.get("limit", [])
        sections.append(
            "FOODS TO LIMIT (reduce but okay in moderation):\n"
            + ("\n".join("- " + f for f in limit) if limit else "- None")
        )
        
        if intent.preferences:
            sections.append(f"DESIRED INGREDIENTS / PREFERENCES: {intent.preferences}")
        
        sections.append(
            "Based on the above, recommend specific food ingredients that are safe and healthy.\n"
            "Provide nutritional breakdown for each recommendation."
        )
        
        return "\n\n".join(sections)


# Create the pipeline
pipeline = NutritionPipeline(
    intent_parser=intent_parser,
    medical_rag=medical_rag,
    nutrition_rag=nutrition_rag,
    safety_filter=safety_filter
)

print("\nNutrition Pipeline ready!")


Nutrition Pipeline ready!


---
## 7. Test the Full Pipeline

In [22]:
# Test the full pipeline end-to-end
test_query = "I have parkinson. I want to make something with chicken, eggs and salad. Quick breakfast. I can't eat tomatoes or onions."

result = pipeline.process(test_query)
result.display()


PROCESSING QUERY...
Query: I have parkinson. I want to make something with chicken, eggs and salad. Quick breakfast. I can't eat tomatoes or onions.

[Step 1] Intent Parser — extracting structured data...
  → Health conditions: parkinson
  → Preferences: chicken, eggs, salad
  → Restrictions: (none)
  → Allergies: ['tomatoes', 'onions']

[Step 2] Medical RAG — getting nutrition constraints...
  → Constraints: ['sugar_g', 'sodium_mg', 'fiber_g', 'saturated_fat_g']
  → Avoid: []
  → Limit: ['processed foods', 'added sugars', 'salt']

[Step 3] Building augmented query...
  → Query built (717 chars)

[Step 4] Nutrition RAG — getting LLM recommendation...
  → Recommendation received

[Step 5] Safety Check — filtering unsafe foods...
  → Searched 10 candidates

[DEBUG] Filtering 10 candidates
[DEBUG] Allergies: ['tomatoes', 'onions']
[DEBUG] Avoid foods: []
  → 10 safe foods after filtering

PIPELINE RESULT

--- Intent ---
UserIntent:
  preferences: chicken, eggs, salad
  health_condition: 